<a href="https://colab.research.google.com/github/masies/CRA/blob/main/pytorchConversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import torch
!pip3 install transformers
import transformers
!pip install sentencepiece
import sentencepiece as spm

from google.colab import auth
auth.authenticate_user()
project_id = 'helical-loop-303918'
bucket_name = 'code_review_automation'

!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://code_review_automation/
gs://t5tutorial1/


In [ ]:
!gsutil cp gs://{bucket_name}/pyTorch_coversion/code/tf_2_pytorch_T5.py ./tf_2_pytorch_T5.py

!mkdir models
!mkdir dumps

!gsutil cp gs://{bucket_name}/model_dumps/model.ckpt-200000.index ./models/model.ckpt-200000.index
!gsutil cp gs://{bucket_name}/model_dumps/model.ckpt-200000.meta ./models/model.ckpt-200000.meta
!gsutil cp gs://{bucket_name}/model_dumps/model.ckpt-200000.data-00000-of-00002 ./models/model.ckpt-200000.data-00000-of-00002
!gsutil cp gs://{bucket_name}/model_dumps/model.ckpt-200000.data-00001-of-00002 ./models/model.ckpt-200000.data-00001-of-00002

!gsutil cp gs://{bucket_name}/pyTorch_coversion/config/config.json ./config.json

Copying gs://code_review_automation/pyTorch_coversion/code/tf_2_pytorch_T5.py...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      
mkdir: cannot create directory ‘models’: File exists
mkdir: cannot create directory ‘dumps’: File exists
Copying gs://code_review_automation/model_dumps/model.ckpt-200000.index...
/ [1 files][  5.5 KiB/  5.5 KiB]                                                
Operation completed over 1 objects/5.5 KiB.                                      
Copying gs://code_review_automation/model_dumps/model.ckpt-200000.meta...
- [1 files][  9.5 MiB/  9.5 MiB]                                                
Operation completed over 1 objects/9.5 MiB.                                      
Copying gs://code_review_automation/model_dumps/model.ckpt-200000.data-00000-of-00002...
/ [1 files][    8.0 B/    8.0 B]                                                
Operat

In [ ]:
!python3 ./tf_2_pytorch_T5.py --tf_checkpoint_path ./models/model.ckpt-200000 --config_file ./config.json --pytorch_dump_path ./dumps

Building PyTorch model from configuration: T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.4.1",
  "use_cache": true,
  "vocab_size": 32128
}

2021-03-17 11:22:50.309715: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Converting TensorFlow checkpoint from /content/models/model.ckpt-200000
Loading TF weight decoder/block_000/layer_000/SelfAttention/k with shape [512, 512]
Loading TF weight decoder/block_000/layer_000/SelfAttention/k_slot_v

In [ ]:
# !gsutil cp gs://{bucket_name}/pyTorch_coversion/code/validation.py ./validation.py
!mkdir core
!mkdir core/__pycache__

!gsutil -m cp -r \
  "gs://code_review_automation/pyTorch_coversion/core/Logger.py" \
  "gs://code_review_automation/pyTorch_coversion/core/__init__.py" \
  "gs://code_review_automation/pyTorch_coversion/core/custom_dataset.py" \
  ./core/

!gsutil -m cp \
  "gs://code_review_automation/pyTorch_coversion/core/__pycache__/Logger.cpython-38.pyc" \
  "gs://code_review_automation/pyTorch_coversion/core/__pycache__/__init__.cpython-38.pyc" \
  ./core/__pycache__

!gsutil cp gs://{bucket_name}/CodeReviewModel/TestModel.model ./TestModel.model
!gsutil cp gs://{bucket_name}/CodeReviewModel/TestModel.vocab ./TestModel.vocab




mkdir: cannot create directory ‘core’: File exists
mkdir: cannot create directory ‘core/__pycache__’: File exists
Copying gs://code_review_automation/pyTorch_coversion/core/Logger.py...
Copying gs://code_review_automation/pyTorch_coversion/core/__init__.py...
Copying gs://code_review_automation/pyTorch_coversion/core/custom_dataset.py...
/ [3/3 files][  2.6 KiB/  2.6 KiB] 100% Done                                    
Operation completed over 3 objects/2.6 KiB.                                      
Copying gs://code_review_automation/pyTorch_coversion/core/__pycache__/Logger.cpython-38.pyc...
Copying gs://code_review_automation/pyTorch_coversion/core/__pycache__/__init__.cpython-38.pyc...
- [2/2 files][  1.4 KiB/  1.4 KiB] 100% Done                                    
Operation completed over 2 objects/1.4 KiB.                                      
Copying gs://code_review_automation/CodeReviewModel/TestModel.model...
/ [1 files][762.0 KiB/762.0 KiB]                                     

In [ ]:
# !gsutil cp ./dumps/pytorch_model.bin gs://code_review_automation/pyTorch_coversion/pyTorch_dumps/pytorch_model.bin

In [ ]:
# model = torch.load("./dumps/pytorch_model.bin")

In [ ]:
!python3 newVal.py --beam_size 1 --batch_size 5 --data_dir ./data/ --tokenizer_name ./TestModel.model --model_name_or_path ./dumps/pytorch_model.bin --config_name ./config.json

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: Calling T5Tokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,
['<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_

In [ ]:
!cat ./results.txt
!rm ./results.txt

[+] Input: public static void <extra_id_0>(String[] args) {System.Out.println("hello world")}
[+] Target: main
	 [-] pred: hello world<< . ("hello world")<< . ("hello world")<< . ("
[+] Input: public static void main(String[] <extra_id_0> {System.Out.println("hello world")}
[+] Target: args)
	 [-] pred: args<< .<<< ("hello world")<<<<<<<<<<<<<<<<
[+] Input: public static void main(String[] args) {System.Out.<extra_id_0>("hello world")}
[+] Target: println
	 [-] pred: println("hello world")<< . ("hello world")<< . ("hello world")<< .
[+] Input: public <extra_id_0> void main(String[] args) {System.Out.println("hello world")}
[+] Target: static
	 [-] pred: .out.println("hello world")<<< world world")<<<<<<<<<<<<<<<<
[+] Input: public static void main(String[] args) {System.<extra_id_0>.println("hello world")}
[+] Target: Out
	 [-] pred: println("hello world")< static void _<<  _<extra_.<<extra_ println("hello world")
[+] Input: Bind<extra_id_0> to the supplied collection.@param name
[+] T